# Classify Iris

## Import Libraries

In [7]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets
from typing import Tuple

## Preparing datasets of iris

In [8]:
iris = datasets.load_iris()

## Standardizing input data

In [9]:
def standardize(data: np.ndarray) -> np.ndarray:
    avg = np.average(data)
    std = np.std(data, axis=0)
    return (data - avg) / std


input_data = standardize(iris.data)

## Converting target data to one-hot format

In [16]:
def convert_onehot(data: np.ndarray) -> np.ndarray:
    size = len(data)
    formatted = np.zeros((size, 3))
    for i in range(size):
        formatted[i, data[i]] = 1.0
    return formatted

target_data = convert_onehot(iris.target)

[[1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0.

## Divide dataset into train and test

In [17]:
input_train = input_data[0::2]
input_test  = input_data[1::2]
target_train = target_data[0::2]
target_test = target_data[1::2]

## Implementing Neural Network

In [ ]:
class BaseLayer:
    def __init__(self, n_x: int, n_y: int) -> None:
        self.w = np.random.randn(n_x, n_y)
        self.b = np.random.randn(n_y)

    def update(self, eta: float) -> None:
        self.w -= eta * self.grad_w
        self.b -= eta * self.grad_b

class MiddleLayer(BaseLayer):
    # Adopt ReLU function as activation function
    def forward(self, x: np.ndarray) -> None:
        self.x = x
        self.u = np.dot(x, self.w) + self.b
        self.y = np.where(self.u <= 0, 0, self.u)

    def backward(self, grad_y: np.ndarray) -> None:
        delta = grad_y * np.where(self.u <= 0, 0, 1)
        self.grad_w = np.dot(self.x.T, delta)
        self.grad_b = np.sum(delta, axis=0)
        self.grad_x = np.dot(delta, self.w.T)

class OutputLayer(BaseLayer):
    def forward(self, x: np.ndarray):
        self.x = x
        u = np.dot(x, self.w) + self.b
        self.y = np.exp(u) / np.sum(np.exp(u), axis=1, keepdims=True)

    def backward(self, t: np.ndarray):
        delta = self.y - t
        self.grad_w = np.dot(self.x.T, delta)
        self.grad_b = np.sum(delta, axis=0)
        self.grad_x = np.dot(delta, self.w.T)

class NeuralNetwork:
    def __init__(self, n_i: int, n_m: int, n_o: int) -> None:
        self.middle_layer1 = MiddleLayer(n_i, n_m)
        self.middle_layer2 = MiddleLayer(n_m, n_m)
        self.output_layer  = OutputLayer(n_m, n_o)

    def forward(self, x: np.ndarray):
        self.middle_layer1.forward(x)
        self.middle_layer2.forward(self.middle_layer1.y)
        self.output_layer.forward(self.middle_layer2.y)
    
    def backward(self, t: np.ndarray):
        self.output_layer.backward(t)
        self.middle_layer2.backward(self.output_layer.grad_x)
        self.middle_layer1.backward(self.middle_layer2.grad_x)

    def update(self, eta: float) -> None:
        self.middle_layer1.update(eta)
        self.middle_layer2.update(eta)
        self.output_layer.update(eta)

    def get_error(self, t: np.ndarray, batch_size: int):
        return -np.sum(t * np.log(self.output_layer.y + 1e-7)) / batch_size
    
        
